In [1]:
# !pip install langchain-chroma

In [2]:
import os

os.environ["HF_HOME"] = "/space/hotel/phit/personal/experiments/weights"
os.environ["TORCH_HOME"] = "/space/hotel/phit/personal/experiments/weights"

from typing import List, Optional, Union

from langchain.callbacks import FileCallbackHandler
from langchain.retrievers import ContextualCompressionRetriever, ParentDocumentRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader, JSONLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS , Chroma
# from langchain_chroma import Chroma
from langchain_core.documents import Document
from loguru import logger
from rich import print
from sentence_transformers import CrossEncoder

from unstructured.cleaners.core import clean_extra_whitespace, group_broken_paragraphs

logfile = "log/output.log"
logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)


persist_directory = None




class RAGException(Exception):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


def rerank_docs(reranker_model, query, retrieved_docs):
    query_and_docs = [(query, r.page_content) for r in retrieved_docs]
    scores = reranker_model.predict(query_and_docs)
    return sorted(list(zip(retrieved_docs, scores)), key=lambda x: x[1], reverse=True)


def load_pdf(
    files: Union[str, List[str]] = "../data/cv/Bui Tien Phat resume (1).pdf"
) -> List[Document]:
    if isinstance(files, str):
        loader = UnstructuredFileLoader(
            files,
            post_processors=[clean_extra_whitespace, group_broken_paragraphs],
        )
        return loader.load()

    loaders = [
        UnstructuredFileLoader(
            file,
            post_processors=[clean_extra_whitespace, group_broken_paragraphs],
        )
        for file in files
    ]
    docs = []
    for loader in loaders:
        docs.extend(
            loader.load(),
        )
    return docs

VIETNAMWORKS = ['_id', 'url', 'job_name', 'company_name', 'salary', 'end_date',
       'address', 'posted_date', 'job_function', 'job_industry', 'job_level',
       'skill', 'preferred_language', 'job_description'] # job_requirements

TOPCV = ['_id', 'urls', 'job_name', 'company_name', 'address', 'salary',
       'remaining', 'job_description', 'benefits',
       'application_method', 'level', 'experience', 'number_of_recruitment',
       'work_form', 'gender', 'working_time'] # job_requirements

# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    
    # metadata["urls"] = record.get("urls")
    # metadata["job_name"] = record.get("job_name")
    # metadata["company_name"] = record.get("company_name")
    # metadata["address"] = record.get("address")
    # metadata["salary"] = record.get("salary")
    # metadata["remaining"] = record.get("remaining")
    # # metadata[""] = record.get("Mô tả công việc")
    # metadata["Yêu cầu ứng viên"] = record.get("Yêu cầu ứng viên")
    # metadata["Quyền lợi"] = record.get("Quyền lợi")
    # metadata["Địa điểm làm việc"] = record.get("Địa điểm làm việc")
    # metadata["Cách thức ứng tuyển"] = record.get("Cách thức ứng tuyển")
    # metadata["Cấp bậc"] = record.get("Cấp bậc")
    # metadata["Kinh nghiệm"] = record.get("Kinh nghiệm")
    # metadata["Số lượng tuyển"] = record.get("Số lượng tuyển")
    # metadata["Hình thức làm việc"] = record.get("Hình thức làm việc")
    # metadata["Giới tính"] = record.get("Giới tính")
    
    for key in VIETNAMWORKS:
        metadata[key] = record.get(key)
    
    metadata = {key: f'{" ".join(val[0]) if isinstance(val, list) else val}' for key, val in metadata.items()}

    

    return metadata


def load_jsonl(
    files: Union[str, List[str]] = "../data/crawl/train_test.jsonl"
) -> List[Document]:
    if isinstance(files, str):
        loader = JSONLoader(
            files,
            json_lines=True,
            jq_schema='.',
            content_key="job_requirements", 
            text_content=False,
            metadata_func=metadata_func
        )
        return loader.load()

    loaders = [
        JSONLoader(
            file,
            json_lines=True,
            jq_schema='.messages[]',
            content_key="content",
            metadata_func=metadata_func
        )
        for file in files
    ]
    docs = []
    for loader in loaders:
        docs.extend(
            loader.load(),
        )
    return docs

def create_parent_retriever(
    docs: List[Document], embeddings_model: HuggingFaceBgeEmbeddings()
):
    parent_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n", "\n\n"],
        chunk_size=2000,
        length_function=len,
        is_separator_regex=False,
    )

    # This text splitter is used to create the child documents
    child_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n", "\n\n"],
        chunk_size=1000,
        chunk_overlap=300,
        length_function=len,
        is_separator_regex=False,
    )
    # The vectorstore to use to index the child chunks
    vectorstore = Chroma(
        collection_name="split_documents",
        embedding_function=embeddings_model,
        persist_directory=persist_directory,
    )#.as_retriever()
    
    # vectorstore = Chroma.from_documents(docs, embeddings_model)#.as_retriever()
    print("vectorstore: ", vectorstore)
    # The storage layer for the parent documents
    store = InMemoryStore()
    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
        k=10,
    )
    retriever.add_documents(docs)
    return retriever


def retrieve_context(query, retriever, reranker_model):
    retrieved_docs = retriever.get_relevant_documents(query)

    if len(retrieved_docs) == 0:
        raise RAGException(
            f"Couldn't retrieve any relevant document with the query `{query}`. Try modifying your question!"
        )
    reranked_docs = rerank_docs(
        query=query, retrieved_docs=retrieved_docs, reranker_model=reranker_model
    )
    return reranked_docs

# https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/BGE_M3
def load_embedding_model(
    model_name: str = "BAAI/bge-large-en-v1.5", device: str = "cuda"
) -> HuggingFaceBgeEmbeddings:
    model_kwargs = {"device": device}
    encode_kwargs = {
        "normalize_embeddings": True
    }  # set True to compute cosine similarity
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embedding_model

# https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/llm_reranker
def load_reranker_model(
    reranker_model_name: str = "BAAI/bge-reranker-large", device: str = "cuda"
) -> CrossEncoder:
    reranker_model = CrossEncoder(
        model_name=reranker_model_name, max_length=512, device=device
    )
    return reranker_model


def main(
    file: str = "../data/crawl/train_test.jsonl",
    query: Optional[str] = None,
    llm_name="mistral",
):
    # docs = load_pdf(files=file)
    docs = load_jsonl("../data/crawl/vnw.jsonl")
    # print(docs)

    embedding_model = load_embedding_model()
    retriever = create_parent_retriever(docs, embedding_model)
    reranker_model = load_reranker_model()

    context = retrieve_context(
        query, retriever=retriever, reranker_model=reranker_model
    )[0]
    print("context:\n", context, "\n", "=" * 50, "\n")


# if __name__ == "__main__":
#     # from jsonargparse import CLI

#     # CLI(main)
#     # main(query="What is the job description for Network Engineer?")
#     main(query="What is job description in Da Nang ?")
    

In [3]:
docs = load_jsonl('../data/preprocessed/preprocessed_vnw.jsonl')
# print(docs)

embedding_model = load_embedding_model(model_name="BAAI/bge-m3")
retriever = create_parent_retriever(docs, embedding_model)
reranker_model = load_reranker_model(reranker_model_name="BAAI/bge-reranker-v2-m3")

vectorstore:  <langchain_community.vectorstores.chroma.Chroma object at 0x7ff051fd1930>

In [4]:
import os
files = ["../data/cv/" + file for file in os.listdir("../data/cv")]
files

['../data/cv/CV Cap Tan Dat - Intern Developer.pdf',
 '../data/cv/CV_SonBao_DS.pdf',
 '../data/cv/Bui Tien Phat resume (1).pdf',
 '../data/cv/CV Nguyễn việt hoàng - CV lập trình-TopCV.vn.pdf',
 '../data/cv/CV _ Đinh Tuấn Nam_BE Developer(.NET)_HCM.pdf',
 '../data/cv/CV_MaiHuyKhang_DA.pdf',
 '../data/cv/CV_NGUYEN_MINH_SON.pdf']

In [5]:
cv = load_pdf(files)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

In [10]:
print(cv[3].page_content)

HỒ SƠ CÁ NHÂN

19/01/2003

0387 36 1513

✉ hoangpham19112002@gmail.com

https://github.com/hoangphampvh/ csharp5.git

Nguyễn Cơ Thạch/Mỹ Đình 2/ Hà Nội

CAREER GOALS

Các mục tiêu ngắn hạn:

Củng cố, bổ sung những kiến thức hiện có để hoàn thiện và nâng cao giá trị cá nhân.

Đồng thời, trau dồi các kỹ năng và kiến thức .NET hiện tại, làm việc trong vai trò lập trình viên .NET.

Mục tiêu dài hạn:

Trở thành Nhà phát triển cấp cao có chuyên môn vững vàng trong vòng 3-4 năm kinh nghiệm chuyên môn.

SKILLS

Kỹ năng cứng Đọc và hiểu tài liệu tiếng Anh GitHub, ASP .NET Core, .NET framework, SOLID, design pattern, Redis, 
Bootstrap và MudBlazor Sử dụng khung Bootstrap và MudBlazor để thiết kế giao diện người dùng. HTML, CSS và jQuery 
(KENDO) cho Front-end Kiến thức cơ bản về kiểm thử ứng dụng.

Kỹ năng mềm Làm việc hiệu quả trong các dự án nhóm với kỹ năng hợp tác. Duy trì sự tập trung và kiên trì trong công
việc dưới áp lực và thời hạn

NGUYỄN VIỆT HOÀNG FREHER .NET PROGRAMMER

HỌC VẤN

FPT Polytechnic College

NFORMATION TECHNOLOGY 10/2021 

-

 01/2024

Hiện tại tôi đang học học kỳ thứ 7 với tư cách là sinh viên. Tôi đã đạt được danh hiệu Sinh viên Xuất sắc trong học
kỳ Mùa thu năm 2021 và giành được ba danh hiệu Sinh viên Xuất sắc cho các học kỳ Mùa xuân 2022, Mùa hè 2022 và Mùa 
thu 2022.

KINH NGHIỆM LÀM VIỆC

FPT Polytechnic College

.NET PROGRAMMER 10/2021 

-

 01/2024

Thực hiện các dự án liên quan đến .NET.

Cong ty xxx(3 tháng)

THỰC HIỆN CÁC DỰ ÁN LIÊN QUAN ĐẾN .NET. 24/5 - 24/8

Làm quen và tham gia mô hình phát triển hệ thống phần mềm sau 3 tháng thực tập.

FPT Project Workshop (2 tháng)

DEVELOPER 01/04/2023 

-

 20/6/2023

Mô tả dự án: Phục vụ nhu cầu tổ chức các cuộc thi đánh máy cho học sinh, sinh viên Công nghệ được sử dụng:

Sử dụng EF Core để truy cập dữ liệu. Sử dụng LinQ để truy vấn.

Xây dựng giao diện người dùng bằng Bootstrap và Blazor.

Mã nguồn được quản lý bằng GitHub. Đóng góp chính cho dự án:

Xây dựng biểu đồ kết quả thống kê cho các cuộc thi.

Đã triển khai ủy quyền người dùng và Jwt.

My Project



Mô tả dự án: Xây dựng website thương mại điện tử.

Công nghệ được sử dụng:

chặt chẽ. Đảm bảo công việc được hoàn thành đúng thời gian và chất lượng cao. Quản lý thời gian hiệu quả để hoàn 
thành nhiệm vụ được giao và đạt được mục tiêu. Chủ động trong công việc và học hỏi từ người khác. Chấp nhận những 
lời chỉ trích và phản hồi để cải thiện hiệu suất công việc.

Đã sử dụng ASP.NET Core, Blazor, Bootstrap, Git và máy chủ Redis.

Chức Năng: Xác thực và ủy quyền người dùng (Danh tính).

Chức năng nhập và xuất Excel. Đăng nhập bằng JWT và OAuth2. Xử lý thanh toán bằng chứng từ. Thông báo email tự 
động. Mua và bán sản phẩm. Tích hợp SignalR cho các tính năng thời gian thực. Tích hợp thanh toán VnPay, MoMo, 
ZaloPay

Kiến thức đang và đã học



Docker, RabbitMQ

© topcv.vn

© topcv.vn

In [11]:
# query = "What are some job descriptions for Data?"
query = cv[3].page_content
context = retrieve_context(
    query, retriever=retriever, reranker_model=reranker_model
)[:]
print("context:\n", context, "\n", "=" * 50, "\n")

context:

[
    (
        Document(
            page_content='YÊU CẦU VÀ KINH NGHIỆM, Tốt nghiệp đại học chính quy ngành công nghệ thông tin - phần 
mềm, Có kinh nghiệm lập trình, sử dụng tốt các ngôn ngữ lập trình và có tính logic cao., Ít nhất 2 năm kinh nghiệm 
trên project thực tế., Biết tiếng Anh, hoặc tiếng Pháp là một lợi thế, YÊU CẦU VỀ KỸ THUẬT, Có kinh nghiệm với 
framework Spring Java, JPA, Hibernate., Có kinh nghiệm với webservices: Rest API, Soap API, Có kiến thứ SQL vững, 
từng sử dụng DB Postgres, Sử dụng tốt BOOTSTRAP HTML5, CSS3, JQUERY, AJAX, ANGULARJS, Có hiểu biết về Eclipse, 
Maven, SVN, TÍNH CÁCH CÁ NHÂN, Có tinh thần làm việc nhóm, Thân thiện trong giao tiếp, có tinh thần trách nhiệm 
cao., Sẵn sàng trao đổi ý kiến với mọi người và tìm cách giải quyết công việc hiệu quả, Đam mê lập trình, thích tìm
hiểu và ứng dụng công nghệ mới, ham học hỏi và có tính sáng tạo., Có khả năng giải quyết vấn đề độc lập, Tính kỷ 
luật cao., HỒ SƠ: CV kèm hình và chi tiết những công việc đã hoàn thành cũng như mức lương mong muốn',
            metadata={
                'source': 
'/mnt/net/i2x256-ai03/hotel/phit/personal/applied-data-science/data/preprocessed/preprocessed_vnw.jsonl',
                'seq_num': '71',
                '_id': '661621edbb86ac211a66e982',
                'url': 
'https://www.vietnamworks.com/java-developer-331-1-1-3-1--1751666-jv?source=searchResults&searchType=2&placement=17
51816&sortBy=latest',
                'job_name': 'Java Developer',
                'company_name': 'Datawings',
                'salary': '$750 - $950',
                'end_date': '2024-04-11',
                'address': '61a-63a Vo Van Tan, Q3',
                'posted_date': '12/03/2024',
                'job_function': 'Công Nghệ Thông Tin/Viễn Thông,  > , Phần Mềm Máy Tính',
                'job_industry': 'Phần Mềm CNTT/Dịch vụ Phần mềm',
                'job_level': 'Nhân viên',
                'skill': 'PostgreSQL, Java Spring Framework, IT Phần Mềm, Software Engineering, HTML5',
                'preferred_language': 'Bất kỳ',
                'job_description': '* Làm việc nhóm., * Phát triển chức năng mới của webapp và sửa lỗi trên 
framework Spring / MVC / DB POSTGRES, * Lập kế hoạch triển khai và ước tính thời gian chính xác cho các công việc 
được giao., * Phát hiện, báo cáo những lỗi thường gặp và đưa ra giải pháp, * Đóng góp ý kiến để ứng dụng ngày một 
tốt hơn và dễ sử dụng., * Hỗ trợ với các thành viên khác để hoàn thành công việc.'
            }
        ),
        0.008506765
    ),
    (
        Document(
            page_content='•\\tSinh viên năm cuối hoặc sắp tốt nghiệp chuyên ngành Công nghệ Thông tin hoặc có liên 
quan, •\\tCó kiến thức lập trình một trong các ngôn ngữ lập trình như: Java, .NET, C, ASP.NET, SQL..., •\\tSẵn sàng
học hỏi và có khả năng học hỏi nhanh các công nghệ, ngôn ngữ lập trình mới., •\\tƯu tiên có thể thực tập fulltime, 
Quyền lợi, •\\tĐược đào tạo để xét tuyển lên vị trí chính thức, cơ hội làm việc lâu dài, •\\tĐược học hỏi, đào tạo 
và tham gia vào các dự án phần mềm lớn theo xu hướng công nghệ 4.0 trong lĩnh vực: Chính phủ, Y tế 4.0, SmartCity, 
Doanh nghiệp, Ngân hàng..., •\\tĐược đào tạo, hướng dẫn bởi các chuyên gia hàng đầu của FPT Information System, 
•\\tĐược tham gia các hoạt động văn hoá, sự kiện hấp dẫn của Công ty FPT IS và Tập đoàn FPT., •\\tĐược xem xét 
tuyển chính thức sau giai đoạn đào tạo, Địa điểm làm việc, - Hồ Chí Minh: Lô B3, đường Sáng Tạo, khu E-Office, Khu 
chế xuất Tân Thuận, Quận 7, Thời gian làm việc, Thứ 2 - Thứ 6 (từ 08:00 đến 17:00)',
            metadata={
                'source': 
'/mnt/net/i2x256-ai03/hotel/phit/personal/applied-data-science/data/preprocessed/preprocessed_vnw.jsonl',
                'seq_num': '703',
                '_id': '6628a4b38a759f0de047ef95',
                'url': 
'https://www.vietnamworks.com/thuc-tap-sinh-lap-trinh-java-net-sql-1768764-jv?source=searchResults&searchType=2&pla
cement=1768764&sortBy=latest',
          